In [1]:
import pandas as pd
import json
import configparser
from pathlib import Path
import os

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import SimpleRNN

configs = configparser.ConfigParser()
configs.read("configuracoes.conf",encoding='utf-8')
data_folder = Path(configs["DEFAULT"]["folder_path"])
path = data_folder / configs["DEFAULT"]["file"]
objeto = configs["DEFAULT"]["objeto"]
regiao = configs["DEFAULT"]["regiao"]

look_back = 12 #Quantidade de meses anteriores usado para prever
forecast_horizon = int(configs["DEFAULT"]["meses"]) #Quantidade de meses a serem previstos

# Prever Cesta

In [2]:
#Carrega os dados
cesta = pd.read_excel(path)
cesta.drop(['ano'], axis=1, inplace=True)
cesta = cesta / 1000
cesta = cesta.iloc[::, ::]

#Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
for n_step in range(1, look_back + forecast_horizon):
    cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
cesta.dropna(inplace=True)
cesta.reset_index(drop=True, inplace=True)

#Divide os dados em Treino e Teste
X_train = cesta.iloc[::, :look_back:].values
y_train = cesta.iloc[::, look_back::].values
X_val = cesta.iloc[-1::, -12::].values

#remodela os dados para formato (batches, sequencia de tempo, features)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

#Cria o modelo
model=Sequential()
model.add(SimpleRNN(24, stateful=True, input_shape=(look_back, 1), batch_input_shape=(1, look_back, 1)))
model.add(Dense(forecast_horizon))
model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

#Treina e prever (forecast_horizon) quantidade de tempo
history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
forecast = model.predict(X_val, batch_size=1)

resultados = [valor for valor in forecast[0]]
objeto = objeto.replace(" ",'_')
output = {objeto.lower():str(resultados)}
filename = f"previsao_RNN_{objeto.lower()}_{regiao.lower()}.json"
pasta_atual = os.getcwd()
full_url = Path(pasta_atual) / 'resultados' / 'previsoes_cesta' / filename
os.makedirs(os.path.dirname(full_url), exist_ok=True)
with open(full_url,"w") as file:
    json.dump(output,file,ensure_ascii=False)
print("Arquivo com resultados gerados")

Epoch 1/150
200/200 [==============================] - 0s 1ms/step - loss: 0.0016
Epoch 2/150
200/200 [==============================] - 0s 1ms/step - loss: 0.0010
Epoch 3/150
200/200 [==============================] - 0s 1ms/step - loss: 7.6036e-04
Epoch 4/150
200/200 [==============================] - 0s 1ms/step - loss: 6.6750e-04
Epoch 5/150
200/200 [==============================] - 0s 1ms/step - loss: 6.6341e-04
Epoch 6/150
200/200 [==============================] - 0s 1ms/step - loss: 6.0544e-04
Epoch 7/150
200/200 [==============================] - 0s 1ms/step - loss: 6.3098e-04
Epoch 8/150
200/200 [==============================] - 0s 1ms/step - loss: 5.7354e-04
Epoch 9/150
200/200 [==============================] - 0s 1ms/step - loss: 5.9464e-04
Epoch 10/150
200/200 [==============================] - 0s 1ms/step - loss: 5.5989e-04
Epoch 11/150
200/200 [==============================] - 0s 1ms/step - loss: 5.6122e-04
Epoch 12/150
200/200 [==============================] - 0s 1

# Prever Produtos

In [2]:
path_inicial = os.getcwd() + "\\datasets_cestas\\datasets_produtos\\"

regioes = ['ilheus','itabuna']
produtos = ['acucar', 'arroz', 'banana', 'cafe', 'carne', 'farinha', 'feijao', 'leite', 
            'manteiga', 'oleo', 'pao', 'tomate']

for regiao in regioes:
    for objeto in produtos:
        #Carrega os dados
        path = f'{path_inicial}{regiao}/{objeto}_{regiao}.xlsx'
        cesta = pd.read_excel(path)
        cesta.drop(['ano'], axis=1, inplace=True)
        cesta = cesta / 1000
        cesta = cesta.iloc[::]

        #Cria sequencias de tempo de tamanho (meses anteriores + meses a serem previstos)
        for n_step in range(1, look_back + forecast_horizon):
            cesta[f'preco t(h + {n_step})'] = cesta['preco'].shift(-n_step).values
        cesta.dropna(inplace=True)
        cesta.reset_index(drop=True, inplace=True)

        #Divide os dados em Treino e Teste
        X_train = cesta.iloc[::, :look_back:].values
        y_train = cesta.iloc[::, look_back::].values
        X_val = cesta.iloc[-1::, -12::].values

        #remodela os dados para formato (batches, sequencia de tempo, features)
        X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
        X_val = X_val.reshape((X_val.shape[0], X_val.shape[1], 1))

        #Cria o modelo
        model=Sequential()
        model.add(SimpleRNN(24, stateful=True, input_shape=(look_back, 1), batch_input_shape=(1, look_back, 1)))
        model.add(Dense(forecast_horizon))
        model.compile(loss='mean_squared_error', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0003))

        #Treina e prever (forecast_horizon) quantidade de tempo
        history = model.fit(X_train, y_train, epochs= 150, batch_size=1, shuffle=False)
        forecast = model.predict(X_val, batch_size=1)

        resultados = [valor for valor in forecast[0]]
        objeto = objeto.replace(" ", '_')
        output = {objeto.lower():str(resultados)}
        filename = f"previsao_RNN_{objeto.lower()}_{regiao.lower()}.json"
        pasta_atual = os.getcwd()
        full_url = Path(pasta_atual) / 'resultados' / 'previsoes_produtos' / f'{regiao}' / filename
        os.makedirs(os.path.dirname(full_url), exist_ok=True)
        with open(full_url,"w") as file:
            json.dump(output,file,ensure_ascii=False)
        print("Arquivo com resultados gerados")

Epoch 1/150
200/200 [==============================] - 0s 1ms/step - loss: 1.1224e-05
Epoch 2/150
200/200 [==============================] - 0s 1ms/step - loss: 4.7230e-06
Epoch 3/150
200/200 [==============================] - 0s 1ms/step - loss: 5.4554e-06
Epoch 4/150
200/200 [==============================] - 0s 1ms/step - loss: 4.6866e-06
Epoch 5/150
200/200 [==============================] - 0s 1ms/step - loss: 4.7365e-06
Epoch 6/150
200/200 [==============================] - 0s 1ms/step - loss: 4.6014e-06
Epoch 7/150
200/200 [==============================] - 0s 1ms/step - loss: 4.4381e-06
Epoch 8/150
200/200 [==============================] - 0s 1ms/step - loss: 4.2930e-06
Epoch 9/150
200/200 [==============================] - 0s 1ms/step - loss: 4.1734e-06
Epoch 10/150
200/200 [==============================] - 0s 1ms/step - loss: 4.0695e-06
Epoch 11/150
200/200 [==============================] - 0s 1ms/step - loss: 3.9453e-06
Epoch 12/150
200/200 [==============================